# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install metasynth`

In [1]:
# %pip install metasynth

In [2]:
# import required packages
import datetime as dt
import polars as pl
from metasynth import MetaDataset, demo_file

## Step 1: Transforming your data into a polars DataFrame

The first step in creating the metadata is reading and converting your dataset to a polars DataFrame. 

In [3]:
demonstration_fp = demo_file()
df = pl.read_csv(demonstration_fp, try_parse_dates=True, dtypes={
    "Sex": pl.Categorical,
    "Embarked": pl.Categorical})
df.head()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],str
1,"""Braund, Mr. Ow…","""male""",22,0,7.25,null,"""S""",1937-10-28,15:53:04,2022-08-05 04:43:34,null
2,"""Cumings, Mrs. …","""female""",38,0,71.2833,"""C85""","""C""",null,12:26:00,2022-08-07 01:56:33,null
3,"""Heikkinen, Mis…","""female""",26,0,7.925,null,"""S""",1931-09-24,16:08:25,2022-08-04 20:27:37,null
4,"""Futrelle, Mrs.…","""female""",35,0,53.1,"""C123""","""S""",1936-11-30,null,2022-08-07 07:05:55,null
5,"""Allen, Mr. Wil…","""male""",35,0,8.05,null,"""S""",1918-11-07,10:59:08,2022-08-02 15:13:34,null


Now, let's check the data types of our DataFrame:

In [4]:
dict(zip(df.columns, df.dtypes))

{'PassengerId': Int64,
 'Name': Utf8,
 'Sex': Categorical,
 'Age': Int64,
 'Parch': Int64,
 'Fare': Float64,
 'Cabin': Utf8,
 'Embarked': Categorical,
 'Birthday': Date,
 'Board time': Time,
 'Married since': Datetime(time_unit='us', time_zone=None),
 'all_NA': Utf8}

We see that most variables are now nicely specified as strings, categories, dates and ints where necessary.

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [5]:
meta_dataset = MetaDataset.from_dataframe(df)

/Users/qubix/Documents/work/metasynth/metasynth/metasynth/provider.py:199: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [6]:
print(meta_dataset)

# Rows: 891
# Columns: 12

{'name': 'PassengerId', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.0, 'distribution': "{'implements': 'core.discrete_uniform', 'provenance': 'builtin', 'class_name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'string', 'dtype': 'Utf8', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'categorical', 'dtype': 'Categorical', 'prop_missing': 0.0, 'distribution': "{'implements': 'core.multinoulli', 'provenance': 'builtin', 'class_name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'implements': 'core.discrete_uniform', 'provenance': 'builtin', 'class_name': 'DiscreteUniformDistribution', 'p

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [7]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [8]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,f32,cat,date,time,datetime[μs],f32
238,"""$1H3mrvWcFvW>4…","""female""",67,1,30.093937,null,"""Q""",1930-01-05,12:31:25,2022-08-15 01:35:55,null
808,"""puoj ^?}u_);SQ…","""female""",null,0,1.572982,null,"""S""",1911-09-25,12:05:16,2022-07-24 14:20:05,null
571,"""eI+>k I'S<gs P…","""female""",61,0,66.063047,null,"""S""",1940-03-12,null,2022-08-10 02:14:20,null
53,""" k62+Ij*4hks%p…","""male""",null,0,0.302661,null,"""C""",1920-06-17,null,2022-07-17 16:29:17,null
833,"""hYcHE|hm,mQ.0S…","""female""",35,1,20.32786,null,"""C""",1933-02-26,11:54:27,2022-07-25 10:34:40,null


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [9]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'Int64',
 'prop_missing': 0.0,
 'distribution': {'implements': 'core.unique_key',
  'provenance': 'builtin',
  'class_name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [10]:
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f32
1,"""y^dpnvg8""XE(FP…","""male""",24,0,16.383339,null,"""S""",1914-11-20,15:48:40,2022-07-31 03:53:33,null
2,"""$L?0zKsN+C='O4…","""male""",36,1,28.624349,null,"""S""",1924-12-02,12:36:07,2022-07-18 13:18:52,null
3,"""Odi0i*}C0m9)<W…","""male""",null,0,6.618381,null,"""C""",null,null,2022-08-07 20:27:11,null
4,"""lX <:voFlq]h@=…","""male""",72,2,11.135868,null,"""S""",1918-04-27,14:12:48,2022-08-02 16:28:51,null
5,"""uD\,Cy#2pi rH3…","""male""",62,1,141.462069,"""AeFX""","""S""",1910-03-20,null,2022-08-10 18:31:35,null


Now we that the `PassengerId` column is correctly represented with increasing id numbers.

### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [11]:
# First, we create a specification dictionary for the variables
from metasynth.distribution import FakerDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": FakerDistribution("name")}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f32
1,"""Denise Smith""","""male""",80,1,19.745816,null,"""C""",1912-03-27,18:03:23,2022-07-29 03:11:55,null
2,"""Sarah James""","""female""",57,0,14.240361,"""T|K&k :QX""","""S""",1918-02-22,13:18:24,2022-07-17 06:15:29,null
3,"""Thomas Oliver""","""male""",70,0,23.827499,null,"""S""",1904-05-07,13:24:53,null,null
4,"""Kimberly Randa…","""female""",12,0,7.388182,null,"""C""",1905-06-14,13:40:48,2022-07-19 18:44:26,null
5,"""Kathleen Reill…","""female""",42,0,1.015004,null,"""C""",1935-08-26,13:44:49,2022-08-13 14:59:58,null


That already looks a lot better for the `Name` column!

### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [12]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": FakerDistribution("name")},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit an exponential distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f32
1,"""Denise Smith""","""male""",33,0,0.188708,"""FS{P""","""S""",1921-12-01,12:41:34,2022-07-16 12:17:32,null
2,"""Sarah James""","""male""",25,0,0.459442,null,"""S""",1929-08-06,15:34:47,2022-08-06 04:17:38,null
3,"""Thomas Oliver""","""male""",34,0,7.043431,"""FA@e9]4F2J'""","""S""",1929-07-26,13:24:31,2022-07-24 15:10:43,null
4,"""Kimberly Randa…","""male""",27,0,0.682256,null,"""C""",1912-10-05,15:12:25,2022-07-15 13:27:17,null
5,"""Kathleen Reill…","""male""",28,0,0.256727,null,"""C""",1930-08-14,12:38:52,2022-08-05 07:29:42,null


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [13]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[ABCDEF]\d{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": FakerDistribution("name")},
    "Fare": {"distribution": "ExponentialDistribution"}, # estimate / fit an exponential distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(10)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,f64,str,cat,date,time,datetime[μs],f32
1,"""Denise Smith""","""female""",29,0,44.372505,null,"""S""",1907-06-11,10:50:55,2022-07-22 19:35:36,null
2,"""Sarah James""","""male""",24,0,35.821083,null,"""S""",1918-01-28,13:07:14,2022-07-18 00:44:24,null
3,"""Thomas Oliver""","""female""",21,0,60.5651,null,"""Q""",1937-07-31,18:16:35,2022-07-21 15:15:27,null
4,"""Kimberly Randa…","""male""",29,0,9.463605,null,"""S""",1929-07-13,12:03:07,2022-07-26 16:50:49,null
5,"""Kathleen Reill…","""female""",33,0,11.661819,null,"""C""",1932-11-27,18:36:59,2022-08-01 02:39:56,null
6,"""James Sexton""","""male""",23,1,5.705814,"""E108""","""C""",1905-12-15,14:10:42,2022-07-19 16:26:28,null
7,"""Richard Smith""","""female""",32,0,40.989583,null,"""S""",1903-12-07,17:06:55,2022-07-19 14:33:52,null
8,"""Rachel Brooks""","""male""",29,0,32.674552,"""D175""","""S""",1903-08-05,12:39:56,2022-07-16 11:48:45,null
9,"""Diane Perry""","""male""",34,1,55.406916,null,"""C""",null,15:40:15,2022-07-26 18:17:59,null


## Comparing the final synthetic dataset to the original

Let's first compare the averages of the numerical columns:

In [14]:
df.mean()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
f64,str,cat,f64,f64,f64,str,cat,date,time,datetime[μs],str
446.0,null,null,29.693277,0.381594,32.204208,null,null,null,null,null,null


In [15]:
meta_dataset.synthesize(len(df)).mean()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
f64,str,cat,f64,f64,f64,str,cat,date,time,datetime[μs],f32
446.0,null,null,29.493075,0.42312,32.710142,null,null,null,null,null,null


Then, we can also see how many missing values are in each column

In [16]:
df.null_count()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,177,0,0,687,2,78,79,92,891


In [17]:
meta_dataset.synthesize(len(df)).null_count()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,185,0,0,700,1,96,76,95,891
